##### Dizem que um programador Lisp não se pergunta "Como posso resolver esse problema em Lisp?", mas sim "Qual é a melhor linguagem para resolver esse problema?".

# Um breve resumo da aula anterior

Relembrando alguns conceitos:

### Captura de símbolos
* *Livre*: Um símbolo é dito livre quando ele está presente em uma expressão sem ser vinculado a nenhum valor no corpo;
* *Esqueleto*: O esqueleto de uma expansão de macro é toda a expansão, exceto toda aparição dos argumentos passados para o macro;
* *Capturável*: Um símbolo é capturável se, durante a expansão de um macro ele:
  1. ocorre livre no esqueleto;
  2. é vinculado por alguma parte do esqueleto no qual os argumentos passados para o macro são ou vinculados ou computados.

Por exemplo, no macro:

In [ ]:
(define-macro (for-1 vvs . body)
  (let ([var  (car vvs)]
        [val  (cadr vvs)]
        [stop (caddr vvs)])
    `(do ((,var ,val (1+ ,var))
          (limit ,stop))
         ((> ,var limit))
       ,@body)))

Tudo aparenta estar ok, veja:

In [ ]:
(for-1 (x 1 5)
       (display "x: ")
       (display x)
       (newline))

Mas é só alterarmos `x` para `limit` que tudo quebra:

In [ ]:
(for-1 (limit 1 5)
       (display "limit: ")
       (display limit)
       (newline))

Isso ocorre pois o macro `for-1` captura o símbolo `limit` na sexta linha: `(limit ,stop))`

Vimos que, para esses casos, o **LISP** possui uma solução minimamente deselegante, um função chamda `gensym` que gera um símbolo aleatório **não utilizado** no programa:

In [ ]:
(gensym)

In [ ]:
(gensym "Oie")

In [ ]:
(gensym "o i e")

Sendo assim, podemos reescrever `for-1` como:

In [ ]:
(define-macro (for-1 vvs . body)
  (let ([var    (car vvs)]
        [start  (cadr vvs)]
        [stop   (caddr vvs)]
        [glimit (gensym "limit")])
    (format #t "GLIMIT: ~A\n" glimit)
    `(do ((,var ,start (1+ ,var))
          (,glimit ,stop))
         ((> ,var ,glimit))
       ,@body)))

In [ ]:
(for-1 (limit 1 5)
       (display "limit: ")
       (display limit)
       (newline))

Dentro do bloco `let` definimos a variável `glimit` com o valor `(gensym "limit")`, a partir disso utilizamos o **valor** de `glimit` (`,glimit = limit<rand>`) ao invés de um símbolo como `limit`, evitando assim uma captura não intencional de símbolos.

### Questão
Existe um bug no macro a seguir? Se sim, arrume-o!

In [ ]:
(define-macro (nif expr neg zero pos)
  `(let ([result ,expr])
     (cond [(positive? result) ,pos]
           [(negative? result) ,neg]
           [(zero?     result) ,zero])))

### Captura intencional

Vimos também que as vezes queremos construir macros que capturam um símbolo propositalmente, como o `cond` faz com o símbolo `else`

Por exemplo, podemos escrever um macro anafórico `aif`, que transforma o resultado de seu teste no símbolo `it`, permitindo reuso sem a criação explicita de uma nova váriavel:

In [ ]:
(define-macro (aif test then else)
  `(let ([it ,test])
     (if it ,then ,else)))

(aif (and #t 2 3 (+ 5 6))
     (* it it)
     #f)

In [ ]:
(aif (and #t 2 3 #f)
     (it it it)
     (not it))

### Questão
Defina um macro `alambda` que é um `lambda` anafórico, o símbolo `recur` representa o nome da função.

No geral ele funcional como nosso `nlamda`, porém, não permite o usuário dar o próprio nome a função, ao invés ele a nomeia como `recur`.

In [ ]:
(define-macro (alambda args . body)
  ...)       

In [ ]:
(map (alambda (x) (if (= x 0)
                      1
                      (* x (recur (1- x))))) 
     '(1 2 3 4 5))

;; => (1 2 6 24 120)

# Macros Higiênicos
Podemos dizer, de certa forma, que o sistema de macros que apresentei até o momento é **imperativo**, você recebe um conjunto de dados, trata eles e retorna outro dado, sabe-se de antemão o que quer e o que vai ser gerado. 

`gensym` é uma boa prova disso, você conhece que um é símbolo capturável e evita isso utilizando uma função, depois utiliza esse símbolo gerado, explicitamento, no "*código*" retornado.

Além disso, estamos a todo momento manipulando uma sintaxe como uma lista, sabemos que para a chamada `(macro arg1 arg2 ...)` vamos receber uma lista no formato `(arg1 arg2 ...)`, tratamos esses `args` e devolmemos outra lista...

A todo momento nós pensamos *como transformar x em y?*, e isso é uma pergunta que se faz em linguagens **imperativas**!. Em linguagens declarativas só nos importamos com o resultado, pensamos *nesse macro recebo x e devolvo y*, independente do processo (do **como**).

Desta maneira, **Scheme** traz consigo um outro sistema de macros.

A verdade mesmo é que `defmacro` e `define-macro` nem existem em outros **Schemes**, é algo que está no **Guile** por razões históricas, mas chamamos esse estilo de macro de "Estilo do Common Lisp" (mesmo sendo adotado por grande parte dos **LISPs**).

Aqui, queremos mudar a forma de pensar, não mais em listas mas em **síntaxe**, por conta disso, o nome do macro que vamos utilizar na construção de macros higiênicos é `define-syntax`.

### Questão
Tendo como base somente o que você viu até aqui, elabore uma explicação para o uso da palavra *higiênico*? Não existe resposta certa! É só pra acordar vocês um pouco.

### O problema da higiene

Dizemos que um sistema de macros é higiênico quando ele mantêm **transparência referencial**.

Em linguística, dizemos que um contexto possui transparência referencial quando podemos alterar uma expressão por outra com mesmo significado sem alterar a "verdade" (seu significado) da sentença.
Por exemplo:

"Na última segunda-feira, ocorreu uma tempestade em Campinas."

Pordemos substituir *Na última segunda feira* por *No dia 22 de Setembro de 2025* e o significado da sentença será o mesmo:

"No dia 22 de setembro de 2025, ocorreu uma tempestade em Campinas."

Podemos ir além, modificar toda a frase e mesmo assim manter o mesmo resultado:

"No dia 22 de setembro de 2025, houve uma chuva com ventos fortes na cidade onde a Unicamp está localizada."

Essa frase compartilha somente a palavra **uma** com a frase original e, mesmo assim, possui o mesmo significado!

### E o que isso tem a ver com macros???

Perceba que, captura de váriavel ou símbolo é uma consequência direta de um sistemas de macros que não garante a **transparência referencial**, por exemplo, no macro `for-1`, a simples substituição de `x` por `limit` fez com que, a substituição de `var` por seu "significado" `,var = limit` alterou o significado da sentença:

```scheme
(for-1 (limit 1 5)
       (display "limit: ")
       (display limit)
       (newline))
```


### Questão
É possível criar macros anafóricos e manter transparência referencial?

# syntax-rules

`define-syntax` não anda sozinho, ele vem acompanhado com um pattern-matching muito poderoso, o `syntax-rules`, sua sintaxe é:

```scheme
(syntax-rules (<literals>)
   (<pattern> <template>)
   ...)
```

O que isso significa? 

Macros `syntax-rules` permite que o mesmo macro retorne diferentes **sintaxes**, veja o exemplo de um `or`:

In [ ]:
(define-syntax my-or
  (syntax-rules ()
    [(my-or)                ; primeiro caso: zero argumentos
     #t]                    ; síntaxe retornada 
    
    [(my-or exp)            ; segundo caso: 1 argumento
     exp]                   ; síntaxe retornada

    [(my-or exp rest ...)   ; terceiro caso: 2 ou mais argumentos (o uso de `...` será explicado em breve)
     (let ([temp exp])      ; síntaxe retornada
       (if temp
           temp
           (my-or rest ...)))]))

(my-or #f #f #f "sou verdadeiro")

Perceba que em momento algum eu tive que declarar, explicitamente, como algo seria feito, eu "disse": "*caso o argumento do macro tenha formato **x** retorne a síntaxe **y***"

Além disso, não existe nenhuma necessidade de diferenciar o que é feito em tempo de compilação do que o que é "retornado", o sistema de macros higiênicos decide o melhor caminho a partir do pattern-matching.

##### Nota:

Os três pontos (**...**) são chamados **reticências** e, como em português, indicam uma repetição de 0 ou mais items seguindo o padrão anterior, ou seja, nesse caso,
`(my-or exp rest ...)` indica que `(my-or` deve corresponder a esse padrão se ele receber uma `exp` e um ou mais `rest`.

Mas ele é mais poderoso que isso, em seu segundo uso `(my-or rest ...` ele sabe exatamente que a primeira reticências "capturou" o resto do corpo e, conhecendo isso, substitui exatamente o que queremos! Totalmente declarativo.

Por fim veja outra mágica:

In [ ]:
(define temp #f)

(my-or temp temp temp temp temp temp)

Temp não foi capturado, e eu nem precisei declarar a geração de símbolos implicitamente, o sistema de macros higiênicos já cuidou de tudo para mim...

### Questão
Utilizando `define-macro` ou `defmacro` implemente uma versão de my-or, sem captura de váriavel!

#### Curiosidade rápida
Similar ao `let` e `letrec` existem o `let-syntax` e `letrec-syntax`, sim, você pode constuir macros "localmente", veja:

In [ ]:
(let-syntax ([my-unless
              (syntax-rules ()
                [(my-unless condition exp ...)
                 (if (not condition)
                     (begin exp ...))])])
  (my-unless #t
    (display "TEM ALGO DE ERRADO!!!!"))
  (string->symbol "SOU UM SÍMBOLO MUITO ESTRANHO, TENHO ATÉ ACENTO"))

In [ ]:
(my-unless #f
  (display "Agora é pra dar certo... Ou não?"))

Assim como utilizamos `letrec` para funções recursivas, utilizamos `letrec-syntax` para **macros recursivos**:

In [ ]:
(letrec-syntax ((my-or
                 (syntax-rules ()
                   [(my-or)
                    #t]
                   [(my-or exp)
                    exp]
                   [(my-or exp rest ...)
                    (let ((t exp))
                      (if t
                          t
                          (my-or rest ...)))])))
  (my-or #f #f #f 42))

Assim como `define` expande para `letrec` ao ser usado localmente, `define-syntax` se expande para `letrec-syntax` ao ser utilizado localmente

### Questão
Crie um macro `my-when` "localmente" que faz o contrário do `my-unless`, ou seja, ele devolve o corpo se a condição for verdadeira.

## Quando `syntax-rules` deixa de ser o suficiente...

`syntax-rules` é magnífico, ele permite a criação de grande parte dos macros que encontramos por ai de forma 100% declarativa, mas ele ofere pouco controle quanto a síntaxe em si, no geral você consegue no máximo retornar uma síntaxe, não pode nem controlar a ordem de computação de forma clara.

Por conta disso, existe uma alternativa um pouco menos declarativa, o `syntax-case`.

Sua síntaxe é:

```scheme
(syntax-case <syntax> (<literals>)
  (<pattern> <guard>* <template>))
```

Bem parecido com `syntax-rules` certo? A única diferença é a introdução de `syntax`. Vamos ver o motivo!

Utilizando `syntax-rules`, escrevemos `my-unless` como:

In [ ]:
(define-syntax my-unless
  (syntax-rules ()
    [(my-unless condition exp ...)
     (if (not condition)
         (begin exp ...))]))

(my-unless #t
  (display "Não vou aparecer"))

Agora, utilizando `syntax-case` temos algo bem mais verboso:

In [ ]:
(define-syntax my-unless
  (lambda (x)                ; primeira diferença
    (syntax-case x ()
      [(my-unless condition exp ...)
       #'(if (not condition) ; segunda diferença
             (begin exp ...))]))) 

In [ ]:
(my-unless #f
  (display "Vou aparecer"))

A realidade é que, enquanto `syntax-rules` define um transformador de síntaxe, o `syntax-case` só  define uma forma de **desestruturar** um síntaxe (pattern matching) e construir outra (o que vem após o `#'`).

Alguém ainda precisa fazer o papel de dizer: "Ok, faça x quando receber y", e esse alguém é o `lambda (x)`. Sendo essa uma função de um único argumento (a síntaxe), que passa "adiante" a síntaxe "`x`", construindo assim um transformador de síntaxe.

Por fim, o `#'` é conhecido como `syntax`, e, tal como `'x` expande para `(quote x)`, `#'x` expande para `(syntax x)`

In [ ]:
#'x

In [ ]:
(syntax x)

Sendo `#<syntax x` simplesmente uma representação interna de uma síntaxe abstrata para o compilador.

##### Curiosidade:
No caso do **Guile**, `syntax` é representado diretamente por sua linguagem intermediária, chamda de **tree-il** (Tree  Intermediate Language).

### Questão
Escreve `my-or` utilizando `syntax-case` ao invés de `syntax-rules`.

In [ ]:
(define-syntax my-or
   ...)          

Vocês devem imaginar que, se existe um equivalente ao `quote` para syntax, deve existir um equivalente para `quasiquote`, `unquote` e `unquote-splicing`, certo?

E você estão corretos! E seguimos o mesmo padrão:

* `` #` ``  -> quasisyntax
* `` #, ``  -> unsyntax
* `` #,@ `` -> unsyntax-splicing

Mas perceba, não estamos lidando com listas, mas com **síntaxes**:

In [ ]:
#`(+ 1 2)

In [ ]:
#`(#,+ 1 2)

### Questão
O que os código a seguir retornam?

```scheme
#`(+ #,1 #,2)

```scheme
#`(#,@#'(+ 1 2))

```scheme
#`(#,@'(+ 1 2))

```scheme
(define x 3)

#`(`(,x #,x))

```scheme
(define x 3)

`(#`(,x #,x))

Um bom exemplo de uso é o macro a seguir:

In [ ]:
(define-syntax display-compile-timestamp
  (lambda (x)
    (syntax-case x ()
      ((_)
       #`(begin
          (display "The compile timestamp was: ")
          (display #,(current-time))
          (newline))))))

In [ ]:
(display-compile-timestamp)

##### Curiosidade

Em fato, `syntax-rules` é um macro implementado em função de `syntax-case`:

In [ ]:
(define-syntax syntax-rules
  (lambda (x)
    (syntax-case x ()
      ((_ (k ...) ((keyword . pattern) template) ...)
       #'(lambda (x)
           (syntax-case x (k ...)
             ((dummy . pattern) #'template)
             ...))))))

Existe uma coisa nesse código que não contei para vocês:

Tanto em `syntax-rules` quanto em `syntax-case` não é necessário o repetir "a chamada" do macro, por conta disso, na terceira linha usa-se um placeholder `_`, sendo esse o padrão mais utilizado em Scheme.

Pelo mesmo motivo, `keyword` é definido na quarta linha, mas `dummy` é utilizado na sétima linha, esse identificador é utilizado unicamente no código e ignorado em tempo de compilação!

Ou seja, `my-or` poderia ser:

In [ ]:
(define-syntax my-or
  (syntax-rules ()
    [(_)
     #t]
    [(_ exp)
     exp]
    [(_ exp rest ...)
     (let ([t exp])
       (if t
           t
           (my-or rest ...)))]))

(my-or #f #f #f #f #t)

Antes de prosseguirmos irei revelar mais uma mentira que contei 😞:

Somente `(unless condition exp ...)` não é o suficiente, visto que, na verdade, as **reticências** não significam "0 ou mais **a partir** de exp" mas sim "0 ou mais **considerando** o exp", ou seja, o padrão correto seria:  

In [ ]:
(define-syntax my-unless
  (syntax-rules ()
    [(_ conditional exp exp* ...)
     (if conditional
         (begin exp exp* ...))]))

(my-unless #f)

A outra forma mesmo assim da erro pois o `if` não permite um corpo com **0** expressões:

In [ ]:
(if #t
    (begin))

In [ ]:
(define-syntax my-unless
  (syntax-rules ()
    [(_ conditional exp ...)
     (if conditional
         (begin exp ...))]))

(my-unless #f)

Repare que esse erro é o mesmo que o de cima, mas diferente do primeiro!

## Literals

Em ambas as definições, tanto de `syntax-case` quanto `syntax-rules` existe uma lista vazia `()` no final da linha de definição:

```scheme
(syntax-rules ()

(syntax-case x () 
```

Essa lista é reservada parao que chamamos de `literals`.

`Literals` são instrumentos poderosíssimos que nos permite definir símbolos, que serão parte pattern-matching exatamente como são apresentados.

Como assim? Um exemplo fala mais que mil palavras:

In [ ]:
(define-syntax cond1
  (syntax-rules (=> else)
    [(_ test => fun)
     (let ([exp test])
       (if exp (fun exp) #f))]
    [(_ else exp exp* ...)
     (begin exp exp* ...)]
    [(_ test exp exp* ...)
     (if test (begin exp exp* ...))]))

(cond1 10 => (lambda (x) (* x x x)))

In [ ]:
(cond1 else (display "oi") (newline))

### Questão
Desenvolva um macro `and=>` que, tal como o macro acima, permite que o uso do literal `=>` para passar, como argumento de função, o item calculado anteriormente.

Lembrando que, como `and`, ele para no primeiro `#f`. Por exemplo:

```scheme
(and=> 1 2 3 => (lambda (x) (+ x 2)) => even? => not) ;-> #f

(even? (+ 2 3)) #;-> (even? 5) ;-> #f

(not #f) ;-> #t
```

Mas o `not` nunca é aplicado pois o passo retornou `#f`, logo, `and=>` retorna `#f`!

In [ ]:
(define-syntax and=>
  ...)

Perceba que tanto `else` quanto `=>` correspondem diretamente ao seu simbolo, diferente de `test`, `fun`, `exp` e `exp*` que correspondem aos parâmetros do macro

##### Curiosidade
Apesar de, no exemplo, `=>` e `else` serem definidos diretamente no macro, normalmente nós definimos os `literals` como **macros que não correspondem com nada** com o objetivo de exportamos esses símbolos e, quando a biblioteca for importada, evitarmos colisão de nome:

```scheme
(define-syntax else (syntax-rules ()))
(define-syntax => (syntax-rules)))
```

Se o *pattern* não for uma **lista**, **vetor** ou um **identificador** (símbolo), o pattern-matcher vai comparar como um `literal`:

In [ ]:
(define-syntax define-matcher-macro
  (syntax-rules ()
    ((_ name lit)
     (define-syntax name
       (syntax-rules ()
        ((_ lit) #t)       ; <- aqui
        ((_ else) #f))))))

(define-matcher-macro is-literal-foo? "foo")

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;                                  ;;
;;   Vira algo como:                ;;
;;                                  ;;
;;   (define-syntax is-literal-foo? ;;
;;     (syntax-rules ()             ;;
;;       [(_ "foo") #t]             ;;
;;       [(_ else) #f]))            ;;
;;                                  ;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(is-literal-foo? "foo")

(is-literal-foo? "bar")

(let ((foo "foo"))
  (is-literal-foo? foo))

In [ ]:
(is-literal-foo? "bar")

In [ ]:
(let ([foo "foo"])
  (is-literal-foo? foo))

### Por fim...
Podemos agora voltar ao `syntax-case` para melhor entendermos sua necessidade.

Como `syntax-case` são definidos como corpo de funções, nós podemos aproveitar **todo** poder do scheme durante tempo de expansão.

Por exemplo, podemos verificar se um parâmetro é um identificador:

In [ ]:
(define-syntax add1!
  (lambda (x)
    (syntax-case x ()
      ((_ var) (identifier? #'var)
       #'(set! var (add1 var))))))

(define foo 0)

In [ ]:
(add! foo)

In [ ]:
(add! "Não sou um Identificador")

Outra coisa que o `syntax-case` nos permite é a construção de macros anafóricos, pois ele nos permite introduzir vínculos em escopos léxicos (let)

Mas antes ...

### Questão
Por que a implementação a seguir não funciona?

In [ ]:
(define-syntax aif
  (lambda (x)
    (syntax-case x ()
      ((_ test then else)
       #'(let ((it test))
           (if it then else))))))

##### `datum->syntax`
`(datum->syntax template-id datum` é um macro que cria um objeto `syntax` adicionando um `datum` no mesmo contexto léxico que `template-id`... Ele é basicamente um `let` para síntaxe, veja:

In [ ]:
(datum->syntax (syntax oi) 5)

Seu "inverso" é o `syntax->datum` que recebe um objeto `syntax` e retorna seu `datum`, basicamente um `unsyntax`:

In [ ]:
(syntax->datum (syntax oi))

### Questão
Por que a implementação a seguir esta incorreta?

```scheme
(define-syntax aif
  (lambda (x)
    (syntax-case x ()
      ((_ test then else)
       (let ((it (datum->syntax x 'it)))
         #'(let ((it test))
             (if it then else)))))))
```

Uma das soluções corretas seria:

In [ ]:
(define-syntax aif
  (lambda (x)
    (syntax-case x ()
      ((_ test then else)
       (syntax-case (datum->syntax x 'it) ()
         (it
           #'(let ((it test))
               (if it then else))))))))

(aif (getuid) (display it) (display "none")) (newline)

A forma acima funciona, mas é mais feio que bater em mãe.

Mas perceba que existe uma estrutura parecida com o que o `let` faz com um `lambda` ai:

```scheme
(syntax-case (datum->syntax x 'it) ()
  (it 
    <corpo>))
```

Lembraum pouco a transformação de:

```scheme
((lambda (x) <corpo>) 'it)
```

para um `let`:

```scheme
(let ([x it])
  <corpo>)
```

Por conta desse padrão, existe o macro `with-syntax`, que possui a mesma síntaxe de um `let`:

```scheme
(with-syntax ([it (datum->syntax x 'it)])
  <corpo>)
 ```

A melhor solução para se criar um `aif` é:

In [ ]:
(define-syntax aif
  (lambda (x)
    (syntax-case x ()
      ((_ test then else)
       (with-syntax ((it (datum->syntax x 'it)))
         #'(let ((it test))
             (if it then else)))))))

(aif (zero? (random 101))
     (aif (even? it)
          (1+ it)
          (1- it))
     (display "1 a cada 100\n"))

### Questão
Cria o `alambda` utilizando `define-syntax`